In [1]:
%cd
%cd Work/Python/CapsNetwork

import sys

sys.path.append("/Users/paolobonomi/Work/Python/CapsNetwork/src")
       
from model_10_epochs import model
from capsuleNetwork import CapsuleNetwork

/Users/paolobonomi
/Users/paolobonomi/Work/Python/CapsNetwork


 		---- SET UP LOGS ---


Metal device set to: Apple M1


 		---- SET UP DATA SET ---




2022-06-02 11:56:22.497733: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-02 11:56:22.497828: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Instructions for updating:
use `tf.profiler.experimental.start` instead.


 		---- LOAD MODEL ---




 		---- BUILD MODEL ---




2022-06-02 11:56:22.995382: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-02 11:56:22.995398: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-02 11:56:23.471952: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-02 11:56:23.472028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-02 11:56:24.314621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


2022-06-02 11:56:26.732539: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-02 11:56:26.735487: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


In [ ]:
def getAccuracy(model):
    test_sum = 0

    for X_batch, y_batch in testing:
        test_sum += sum(CapsuleNetwork.predict(model, X_batch)==y_batch.numpy())

    print(test_sum/testing_dataset_size)

# getAccuracy(model) # 0.9856

model.summary()